# Exploring Hacker News Posts

In this project, we'll compare two different types of posts from Hacker News, a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either Ask HN or Show HN.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Introduction

First, we'll read in the data and remove the headers.

In [3]:
#Read the data and diplay the list

from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
hn[:5]



[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing Headers from a List of Lists

In [4]:
#Remove the headers
headers = hn[0]
print("Headers: \n", headers)
hn = hn[1:]
hn[:5]

Headers: 
 ['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

The data set contains the title of the post, the number of comments for each post, and the date when each post was created. 

## Separating the Ask HN and Show HN Posts

First, we will identify posts that contain Ask HN or Show HN and separate the data for those types of posts into separate lists. This makes it easier for us the analyze the data.

In [5]:
# Identify posts that begin with 'Ask HN' or 'Show HN' and separate the data into different lists.
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    lower_case_title = title.lower()
    if lower_case_title.startswith("ask hn"):
        ask_posts.append(post)
    elif lower_case_title.startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)
print("No. of Ask posts: ", len(ask_posts))
print("No. of Show posts: ", len(show_posts))
print("No. of other posts: ", len(other_posts))
    

No. of Ask posts:  1744
No. of Show posts:  1162
No. of other posts:  17194


## Calculating the Average Number of Comments for Ask HN and Show HN posts

Now that we separated the ask posts and show posts into different lists, we will find out the average number of comments for each type of post

In [6]:
# Calculating the average number of comments for each type of post
total_ask_posts = 0
for post in ask_posts:
    num_comments = post[4]
    total_ask_posts += int(num_comments)

total_show_posts = 0
for post in show_posts:
    num_comments = post[4]
    total_show_posts += int(num_comments)

avg_ask_posts = total_ask_posts / len(ask_posts)
avg_show_posts = total_show_posts/ len(show_posts)

print("Average no. of comments on Ask posts: ", avg_ask_posts)
print("Average no. of comments on Show posts: ", avg_show_posts)

    

Average no. of comments on Ask posts:  14.038417431192661
Average no. of comments on Show posts:  10.31669535283993


On average, ask posts receive approximately 14 comments, whereas show posts receive approximately 10 comments. Since ask posts are more likely to receive comments, we will focus our analysis on these type of posts

# Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts receive created at each hour of the day.

In [7]:
# Calculate the amount of ask posts created during each hour of day and the number of comments received.
import datetime as dt

list = []
for post in ask_posts:
    list.append([post[6], int(post[4])])


comments_by_hour = {}
counts_by_hour = {}
date_format = "%m/%d/%Y %H:%M"
for row in list:
    num_comments = row[1]
    date_created = row[0]
    hour = dt.datetime.strptime(date_created, date_format).strftime("%H")
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments
    else:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments
print("Counts by hour: \n", counts_by_hour)
print("\nComments by hour: \n", comments_by_hour)
    
    
    


Counts by hour: 
 {'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}

Comments by hour: 
 {'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


# Calculating the Average Number of Comments for Ask HN Posts by Hour

We have the counts by hour and the comments by hour in two separate dictionaries. Now, we are going to calculate the average number of comments by hour

In [13]:
# Calculating the average number of comments by hour

avg_by_hour = []
for hour in comments_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])
    
avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

We have the average number of comments by hour in a list of lists. However, it is hard to identify the hour with the highest number of comments on average. To solve this problem, we will sort the data in descending order.

# Sorting the Data and Displaying the Values 


In [18]:
# Sorting the data in descending order of average comments by hour and displaying it

swap_avg_by_hour = []
for hour, avg in avg_by_hour:
    swap_avg_by_hour.append([avg, hour])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap
    

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [20]:
# Displaying the 5 hours with the highest number of average comments

print("Top 5 hours with the highest number of average comments\n")
for avg, hour in sorted_swap[:5]:
    hour = dt.datetime.strptime(hour, "%H").strftime("%H:%M")
    print(
    "{}: {:.2f} average number of comments".format(hour, avg)
    )

    

Top 5 hours with the highest number of average comments

15:00: 38.59 average number of comments
02:00: 23.81 average number of comments
20:00: 21.52 average number of comments
16:00: 16.80 average number of comments
21:00: 16.01 average number of comments


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There is about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est or 11:00 pm GST (UAE time)

# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (11:00 pm  - 12:00 pm GST).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.